In [ ]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import re

# 종합법률정보 사이트 접속
url = 'https://glaw.scourt.go.kr/wsjo/panre/sjo050.do#//'
driver = webdriver.Chrome('chromedriver') 
driver.get(url)
time.sleep(3)

# 주택임대차보호법 검색
driver.find_element(By.CLASS_NAME, "input_alone").send_keys('주택임대차보호법')
driver.find_element(By.ID, "srch_img").click()
time.sleep(2)

# 요약정보의 첫 번째 글 접속
driver.find_element(By.NAME, "listCont").click()
time.sleep(2)

# 페이지 소스 가져오기
driver.switch_to.window(driver.window_handles[-1])
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
text = soup.get_text()
parts = re.split(r'【판시사항】|【판결요지】|【참조조문】|【참조판례】', text)

judgment = parts[1]
summary = parts[2]
clause = parts[3]

df = pd.DataFrame()

df["판시사항"] = judgment,
df["판결요지"] = summary,
df["참조조문"] = clause

# 다음 글 선택 후 반복 작업

num = 5 # 반복횟수

for i in range(num):
    driver.find_element(By.CSS_SELECTOR, "#header > div.tit_area > a:nth-child(3) > img").click()
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text()
    parts = re.split(r'【판시사항】|【판결요지】|【참조조문】|【.*】', text)

    judgment = parts[1]
    summary = parts[2]
    clause = parts[3]
    
    temp = pd.DataFrame()
    
    temp["판시사항"] = judgment,
    temp["판결요지"] = summary,
    temp["참조조문"] = clause
    
    df = pd.concat([df, temp], axis=0)

df = df.reset_index(drop=True)
df.to_csv("data/raw/precedent.csv", index=False)
df.head()